# 标准调和问题

## 目标和先决条件

包括石化炼油、废水处理和采矿在内的众多行业的企业都使用数学优化来解决调和问题。在本示例中,我们将指导您使用 Gurobi Python API 构建调和问题的混合整数二次约束规划(MIQCP)模型,并向您展示如何使用 Gurobi 优化器为该问题生成最优解。

这个建模示例属于高级水平,我们假设您了解 Python 和 Gurobi Python API,并且具有构建数学优化模型的高级知识。通常,这些示例的目标函数和/或约束条件都很复杂,或需要 Gurobi Python API 的高级功能。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 动机

调和问题是石化炼油、废水处理和采矿行业中的一个具有挑战性的问题。该问题可以看作是最小成本流问题和混合问题的推广。这个问题确实很重要,因为它可以产生显著的成本节约,所以自从 Haverly 在 1978 年指出该问题的非线性结构以来,它就被广泛研究[5]。

---
## 问题描述

最小成本流问题(MCFP)旨在寻找从一组源节点通过可能的中转节点向目标节点发送特定流量的最便宜方式,这些节点构成一个有向容量网络。混合问题是一种只有源节点和目标节点的 MCFP,其中具有不同属性质量的原材料混合在一起以创建最终产品,使得它们的属性质量在允许范围内。

调和问题结合了这两个问题的特点,因为来自不同源的流在中间储池中混合,并在目标节点再次混合。非线性实际上是考虑储池的直接结果,因为储池中给定属性的质量(定义为进入流的质量的加权平均值)是一个未知量,因此需要用决策变量来表示。当网络可以用三分图表示时,我们将这个问题称为标准调和问题,即三个不相交的节点集,同一集合内的节点不相邻。简而言之,可以这样描述:给定一份包含已知属性质量的原材料的源节点列表,在中间储池中混合这些材料以满足多个目标节点的需求和容差的最便宜方式是什么?(Gupte 等人,2017)[4]。文献中存在几种不同的标准调和问题及其扩展的公式,可以分为两大类:一类由流量和质量变量组成,另一类使用流量比例代替质量变量。本笔记本将考虑这两个类别。

---
## 问题实例

作为说明性示例,我们将解决 Rehfeldt 和 Tisljar 在 1997 年提出并被 Audet 等人在 2004 年引用的第二个调和问题:

![图](rehfeldt_tisljar2_graph.png)

![表](rehfeldt_tisljar2_table.png)

为此,让我们声明表示此问题实例所需的数据结构:

In [ ]:
%pip install gurobipy

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# 使用 Gurobi v9.1.0 和 Python 3.7.0 测试

attrs = {'den', 'bnz', 'roz', 'moz'}

sources, cost, supply, content = gp.multidict({
    "s1": [49.2, 6097.56, {'den': 0.82, 'bnz':3, 'roz':99.2,'moz':90.5}],
    "s2": [62.0, 16129, {'den': 0.62, 'bnz':0, 'roz':87.9,'moz':83.5}],
    "s3": [300.0, 500, {'den': 0.75, 'bnz':0, 'roz':114,'moz':98.7}]
})

targets, price, demand, min_tol, max_tol = gp.multidict({
    "t1": [190, 500, {'den': 0.74, 'roz':95,'moz':85}, {'den': 0.79}],
    "t2": [230, 500, {'den': 0.74, 'roz':96,'moz':88}, {'den': 0.79, 'bnz':0.9}],
    "t3": [150, 500, {'den': 0.74, 'roz':91}, {'den': 0.79}]
})

pools, cap = gp.multidict({
    "p1": 1250,
    "p2": 1750
})

# product 函数部署集合 A 和 B 的笛卡尔积
s2p = set(product(sources, pools))
p2t = set(product(pools, targets))
s2t = {("s1", "t2"),
       ("s2", "t1"),
       ("s2", "t3"),
       ("s3", "t1")}

---
## 求解方法

数学规划是一种声明式方法,其中建模者制定一个数学优化模型来捕捉复杂决策问题的关键方面。Gurobi 优化器使用最先进的数学和计算机科学来求解此类模型。

数学优化模型有五个组成部分:

- 集合和索引
- 参数
- 决策变量  
- 目标函数
- 约束条件

一个只涉及不相交变量对的乘积的二次约束通常称为双线性约束,而包含双线性约束的模型通常称为双线性规划。双线性约束是非凸二次约束的特例。这类问题通常使用空间分支定界(sB&B)算法求解。该算法探索整个搜索空间,因此它提供了最优目标值的全局有效下界,并且(在给定足够时间的情况下)它将找到全局最优解(受容差限制)。感兴趣的读者可以参考[参考文献](#references)[3]、[6]和[7]。

我们现在提出两个标准调和问题的替代双线性规划:

### P-公式(浓度)

#### 集合和索引

$G=(V,E)$: 有向图。

$i,j \in V$: 节点集。

$(i,j) \in E \subset V \times V$: 边集。

$N(i)^+ = \{j \in V \mid (i,j) \in E \}$: 从节点 i 接收出流的后继节点集。

$N(j)^- = \{i \in V \mid (i,j) \in E \}$: 向节点 j 发送入流的前驱节点集。

$k \in \text{Attrs}$: 属性集。

$s \in \text{Sources} \subset V$: 源节点集,即 $N(s)^-= \emptyset$。

$t \in \text{Targets} \subset V$: 目标节点集,即 $N(t)^+= \emptyset$。

$p \in \text{Pools} = V \setminus (\text{Sources} \cup \text{Targets})$: 储池集。

#### 参数

$\text{Cost}_s \in \mathbb{R}^+$: 在源节点 s 获取一单位原材料的成本。

$\text{Supply}_s \in \mathbb{R}^+$: 源节点 s 可用的原材料最大单位数。

$\text{Content}_{s,k} \in \mathbb{R}^+$: 源节点 s 的原材料中属性 k 的含量。

$\text{Price}_t \in \mathbb{R}^+$: 在目标节点 t 销售一单位最终混合物的价格。

$\text{Demand}_t \in \mathbb{R}^+$: 目标节点 t 所需的最终混合物最小单位数。

$\text{Min_tol}_{t,k} \in \mathbb{R}^+$: 目标节点 t 最终混合物中属性 k 的最小容差。

$\text{Max_tol}_{t,k} \in \mathbb{R}^+$: 目标节点 t 最终混合物中属性 k 的最大容差。

$\text{Cap}_p \in \mathbb{R}^+$: 储池 p 存储中间混合物的最大容量。

$\text{UB}_{i,j}\in \mathbb{R}^+$: 从节点 i 到节点 j 的最大流量。

#### 决策变量

$\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]$: 从节点 i 到节点 j 的流量。

$\text{quality}_{p,k} \in \mathbb{R}^+$: 储池 p 中属性 k 的浓度。

#### 目标函数

- **利润**: 最大化总利润。

\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\sum_{j \in N(s)^+}{\text{Cost}_s \cdot \text{flow}_{s,j}}}
\tag{0}
\end{equation}

#### 约束条件

- **流量守恒**: 储池 p 的总入流量必须等于其总出流量(不在其中存储任何东西)。

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} - \sum_{s \in N(p)^-}{\text{flow}_{s,p}} = 0 \quad \forall p \in \text{Pools}
\tag{1}
\end{equation}

- **源容量**: 源 s 的总出流量不能超过其容量。

\begin{equation}
\sum_{j \in N(s)^+}{\text{flow}_{s,j}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{2}
\end{equation}

- **储池容量**: 储池 p 的总出流量不能超过其容量。

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{3}
\end{equation}

- **目标需求**: 目标 t 的总入流量必须至少满足其最小需求。

\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{4}
\end{equation}

- **储池浓度**: 储池 p 中属性 k 的浓度表示为与进入流量相关的浓度的加权平均值(注意右侧的双线性项)。

\begin{equation}
\sum_{s \in N(p)^-}{\text{Content}_{s,k} \cdot \text{flow}_{s,p}} = \text{quality}_{p,k} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \quad \forall (p,k) \in \text{Pools} \times \text{Attrs}
\tag{5}
\end{equation}

- **目标容差**: 目标 t 中属性 k 的浓度也是线性混合的结果,并且必须在容差范围内(注意左侧第二个表达式中的双线性项)。

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.1}
\end{equation}

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.2}
\end{equation}

该公式中的双线性项数量与属性数量成正比。另一种公式依赖于表示流量比例而不是浓度的决策变量,因此双线性项不再与属性数量相关。可以使用两种类型的决策变量:

- 储池 p 的总入流量中来自源 s 的比例。
- 储池 p 的总出流量中流向终端 t 的比例。

现在介绍基于第一个选项的模型:

### Q-公式(比例)

#### 决策变量

$\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]$: 从节点 i 到节点 j 的流量。

$\text{prop}_{s,p} \in \mathbb{R}^+$: 储池 p 的总入流量中来自源 s 的比例。

**注意:** 从源到储池的 $\text{flow}$ 变量被 $\text{prop}$ 变量替代。

#### 目标函数

- **利润**: 最大化总利润(注意第二个表达式中的双线性项)。

\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\text{cost}_s \cdot \left( \sum_{t \in N(s)^+ \cap \text{Targets}}{\text{flow}_{s,t}} + \sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} \right) }
\tag{0}
\end{equation}

#### 约束条件

- **源容量**: 源 s 的总出流量不能超过其容量(注意左侧第一个表达式中的双线性项)。

\begin{equation}
\sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} + \sum_{t \in N(s)^+ \cap \text{Targets}}{\\text{flow}_{s,t}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{1}
\end{equation}

- **储池容量**: 储池 p 的总出流量不能超过其容量。

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{2}
\end{equation}

- **目标需求**: 目标 t 的总入流量必须至少满足其最小需求。

\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{3}
\end{equation}

- **储池入流**: 所有进入源对储池 p 的贡献之和必须等于 1。

\begin{equation}
\sum_{s \in N(p)^-}{\text{prop}_{s,p}} = 1 \quad \forall p \in \text{Pools}
\tag{4}
\end{equation}

- **目标容差**: 目标 t 中属性 k 的浓度也是线性混合的结果,并且必须在容差范围内(注意左侧第二个表达式中的双线性项)。

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.1}
\end{equation}

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.2}
\end{equation}

一个缺点是,如果某些源到储池的边有流量容量,我们需要定义额外的约束,而不是仅仅在相关决策变量上指定上限。可以通过双线性项定义这些约束如下:

- **流量限制**: 从源 s 到储池 p 的流量不能超过已安装的容量(注意左侧的双线性项)。在 P-公式中,声明这一点就像设置相关 $\text{flow}$ 变量的上限一样简单。然而,该变量在 Q-公式中不再存在,因此我们需要将容量建模为约束。

\begin{equation}
\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{UB}_{s,p} \quad \forall (i,j) \in E \cap \left( \text{Sources} \times \text{Pools} \right) \mid \text{UB}_{i,j} < \infty
\tag{6}
\end{equation}

---
## Python 实现

使用 Gurobi 求解双线性规划就像配置全局参数 `nonConvex` 一样简单。当将此参数设置为值 2 时,非凸二次问题通过将其转换为双线性形式并应用 sB&B 来求解。我们将首先部署 P-公式模型,然后将其与 Q-公式模型进行比较:

### P-公式(浓度)

In [ ]:
p_pooling = gp.Model("Pooling")

# 设置全局参数
p_pooling.params.nonConvex = 2
p_pooling.params.timelimit = 5*60 # 5分钟的时间限制

# 声明决策变量

# 流量
ik = p_pooling.addVars(s2t, name="Source2Target")
ij = p_pooling.addVars(s2p, name="Source2Pool")
jk = p_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# 质量
prop = p_pooling.addVars(pools, attrs, name="Proportion")

# 部署约束条件集合

# 1. 流量守恒
p_pooling.addConstrs((ij.sum('*',j) == jk.sum(j,'*') for j in pools),
                     name="Flow_conservation")
# 2. 源容量
p_pooling.addConstrs((ij.sum(i,'*') + ik.sum(i,'*') <= supply[i] for i in sources),
                     name="Source_capacity")
# 3. 储池容量
p_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools),
                     name="Pool_capacity")
# 4. 目标需求
p_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets),
                     name="Target_demand")
# 5. 储池浓度
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ij[i,j]
                               for i in sources if (i,j) in s2p)
                      == prop[j,attr]*jk.sum(j,'*') for j in pools for attr in attrs),
                     name="Pool_concentration")
# 6.1 目标(最小)容差
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(prop[j,attr]*jk[j,k]
                                 for j in pools if (j,k) in p2t)
                      >= min_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in min_tol[k].keys()),
                     name="Target_min_tolerances")
# 6.2 目标(最大)容差
p_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(prop[j,attr]*jk[j,k]
                                 for j in pools if (j,k) in p2t)
                      <= max_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in max_tol[k].keys()),
                     name="Target_max_tolerances")

# 部署目标函数

# 0. 总利润
obj = gp.quicksum(price[k]*(ik.sum('*',k) + jk.sum('*',k))
               for k in targets) \
- gp.quicksum(cost[i]*(ij.sum(i,'*') + ik.sum(i,'*'))
           for i in sources)
p_pooling.setObjective(obj, GRB.MAXIMIZE)

# 寻找最优解
p_pooling.optimize()

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter timelimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 24 columns and 38 nonzeros
Model fingerprint: 0xa0238bae
Model has 20 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-02, 1e+02]
  Objective range  [5e+01, 3e+02]
  Bounds range     [8e+02, 8e+02]
  RHS range        [5e+02, 2e+04]

Continuous model is non-convex -- solving as a MIP.

Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 125 rows, 49 columns, 311 nonzeros
Presolved model has 24 bilinear constraint(s)
Variable types: 49 continuous, 0 integer (0 b

此实例的 P-公式有:

- 24 个决策变量
- 10 个线性约束
- 20 个双线性约束  
- 一个线性目标函数

如可以看到,在达到五分钟的时间限制后,我们观察到 103.23% 的差距(此时,当前解的总利润为 411,530.70 美元)。事实上,即使在 20 分钟后,求解器在缩小差距方面也没有取得太大进展。

### Q-公式(比例)

让我们看看 Q-公式模型的表现如何:

In [ ]:
q_pooling = gp.Model("Pooling")

# 设置全局参数
q_pooling.params.nonConvex = 2
q_pooling.params.timelimit = 5*60

# 声明决策变量

# 流量
ik = q_pooling.addVars(s2t, name="Source2Target")
jk = q_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# 比例
p_ij = q_pooling.addVars(s2p, ub=1.0, name="Prop_Source2Pool")

# 部署约束条件集合

# 1. 源容量  
q_pooling.addConstrs((gp.quicksum(p_ij[i,j]*jk.sum(j,'*')
                               for j in pools if (i,j) in s2p)
                      + ik.sum(i,'*') <= supply[i] for i in sources),
                     name="Source_capacity")
# 2. 储池容量
q_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools),
                     name="Pool_capacity")
# 3. 目标需求
q_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets),
                     name="Target_demand")
# 4. 储池入流
q_pooling.addConstrs((p_ij.sum('*',j) == 1 for j in pools),
                     name="Pool_inflow")
# 5.1 目标(最小)容差
q_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(content[i][attr]*p_ij[i,j]*jk[j,k]
                                 for (i,j) in s2p if (j,k) in p2t)
                      >= min_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in min_tol[k].keys()),
                     name="Target_min_tolerances")
# 5.2 目标(最大)容差
q_pooling.addConstrs((gp.quicksum(content[i][attr]*ik[i,k]
                               for i in sources if (i,k) in s2t)
                      + gp.quicksum(content[i][attr]*p_ij[i,j]*jk[j,k]
                                 for (i,j) in s2p if (j,k) in p2t)
                      <= max_tol[k][attr]*(ik.sum('*',k) + jk.sum('*',k))
                      for k in targets for attr in max_tol[k].keys()),
                     name="Target_max_tolerances")

# 部署目标函数

# 0. 总利润
obj = gp.quicksum(price[k]*(ik.sum('*',k) + jk.sum('*',k))
               for k in targets) \
- gp.quicksum(cost[i]*(gp.quicksum(p_ij[i,j]*jk.sum(j,'*')
                             for j in pools if (i,j) in s2p)
                    + ik.sum(i,'*'))
           for i in sources)
q_pooling.setObjective(obj, GRB.MAXIMIZE)

# 寻找最优解
q_pooling.optimize()

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter timelimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 16 columns and 22 nonzeros
Model fingerprint: 0x5737f095
Model has 18 quadratic objective terms
Model has 15 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [6e-01, 1e+02]
  QLMatrix range   [1e-02, 1e+02]
  Objective range  [9e+01, 2e+02]
  QObjective range [1e+02, 6e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 2e+03]
  QRHS range       [5e+02, 2e+04]

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.00s
Presolved: 95 rows, 35 columns, 315 nonzeros
Presolved model has 18 bilinear constraint(s)
Variable types: 35 continuous, 0 integer (0 binary)

Root relaxation:

此实例的 Q-公式有:

- 16 个决策变量
- 7 个线性约束
- 15 个双线性约束
- 一个双线性目标函数

注意,它有更少的决策变量和更少的双线性约束。现在 Gurobi 能够在不到一秒钟的时间内找到 439,182.59 美元的最优解。

---
## 分析

让我们看看找到的最优流量:

### 从源到目标的流量
下表确定了从源节点到目标节点的流量。例如,从源节点 s2 到目标节点 t1 的流量为 966.7 $\times 10^{2}$ 桶。

In [4]:
rows = sources.copy()
columns = targets.copy()
s2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, target in ik.keys():
    if (abs(ik[source, target].x) > 1e-6):
        s2t_plan.loc[source, target] = np.round(ik[source, target].x, 1)
s2t_plan

,t1,t2,t3
s1,0.0,0.0,0.0
s2,966.7,0.0,200.0
s3,0.0,0.0,0.0


### 从储池到目标的流量
下表定义了从储池节点到目标节点的流量。例如,从储池节点 p1 到目标节点 t1 的流量为 92.8 $\times 10^{2}$ 桶。

In [5]:
rows = pools.copy()
columns = targets.copy()
p2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for pool, target in jk.keys():
    if (abs(jk[pool, target].x) > 1e-6):
        p2t_plan.loc[pool, target] = np.round(jk[pool, target].x, 1)
p2t_plan

,t1,t2,t3
p1,92.8,990.6,0.0
p2,1450.0,0.0,300.0


### 从源到储池的流量
下表显示了从源节点到储池节点的流量。例如,从源节点 s2 到储池节点 p1 的流量为 258.3 $\times 10^{2}$ 桶。

In [6]:
rows = sources.copy()
columns = pools.copy()
s2p_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, pool in p_ij.keys():
    if (abs(p_ij[source, pool].x) > 1e-6):
        flow = p_ij[source, pool].x * p2t_plan.loc[pool,:].sum()
        s2p_plan.loc[source, pool] = np.round(flow, 1)
s2p_plan

,p1,p2
s1,325.0,1750.0
s2,258.3,0.0
s3,500.0,0.0


---
## 结论

本笔记本展示了使用 Gurobi 求解双线性规划是多么容易。它还强调了在求解具有挑战性的问题(如标准调和问题)时,替代公式在性能上的显著差异。因此,仔细分析所处理问题的背景,权衡替代模型的优缺点是至关重要的。

---
<a id='references'></a>
## 参考文献

1. Alfaki, M. (2012). 调和问题的模型和求解方法。
2. Audet, C., Brimberg, J., Hansen, P., Digabel, S. L., & Mladenović, N. (2004). 调和问题:替代公式和求解方法。管理科学, 50(6), 761-776。
3. Dombrowski, J. (2015年6月7日). McCormick 包络。来源: https://optimization.mccormick.northwestern.edu/index.php/McCormick_envelopes
4. Gupte, A., Ahmed, S., Dey, S. S., & Cheon, M. S. (2017). 调和问题的松弛和离散化。全局优化期刊, 67(3), 631-669。
5. Haverly, C. A. (1978). 调和问题递归行为的研究。Acm sigmap bulletin, (25), 19-28。
6. Liberti, L. (2008). 全局优化导论。巴黎高等理工学院。
7. Zhuang E. (2015年6月6日). 空间分支定界方法。来源:
https://optimization.mccormick.northwestern.edu/index.php/Spatial_branch_and_bound_method

版权所有 © 2020 Gurobi Optimization, LLC